In [2]:
%pip install ipython-sql
%pip install ipython-sql prettytable
import csv, sqlite3
import prettytable
prettytable.DEFAULT = 'DEFAULT'

con = sqlite3.connect("my_data1.db")
cur = con.cursor()

  Using cached ipython_sql-0.5.0-py3-none-any.whl.metadata (17 kB)
  Using cached sqlparse-0.5.3-py3-none-any.whl.metadata (3.9 kB)
Using cached ipython_sql-0.5.0-py3-none-any.whl (20 kB)
Using cached sqlparse-0.5.3-py3-none-any.whl (44 kB)
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [6]:
# %sql sqlite:///my_data1.db
from sqlalchemy import create_engine
engine = create_engine("sqlite:///my_data1.db")



In [ ]:
import pandas as pd
df = pd.read_csv('dataset_part_2.csv')
df.to_sql("SPACEXTBL", con=engine, if_exists='replace', index=False, method="multi")


90

In [11]:
# DROP THE TABLE IF EXISTS
cur.execute("DROP TABLE IF EXISTS SPACEXTABLE")
con.commit()



Task 1: Display the Names of the Unique Launch Sites

In [16]:
from sqlalchemy import create_engine
import sqlite3

# Create connection to SQLite database
con = sqlite3.connect("my_data1.db")



In [17]:
query = "SELECT name FROM sqlite_master WHERE type='table';"
df_tables = pd.read_sql_query(query, con)
print("Available Tables:\n", df_tables)


Available Tables:
         name
0  SPACEXTBL


In [19]:
import pandas as pd

# Connect to the database
con = sqlite3.connect("my_data1.db")

# Check the table schema
query = "PRAGMA table_info(SPACEXTBL);"
df_columns = pd.read_sql_query(query, con)
print("Columns in SPACEXTBL:\n", df_columns)

Columns in SPACEXTBL:
     cid            name     type  notnull dflt_value  pk
0     0    FlightNumber  INTEGER        0       None   0
1     1            Date     TEXT        0       None   0
2     2  BoosterVersion     TEXT        0       None   0
3     3     PayloadMass     REAL        0       None   0
4     4           Orbit     TEXT        0       None   0
5     5      LaunchSite     TEXT        0       None   0
6     6         Outcome     TEXT        0       None   0
7     7         Flights  INTEGER        0       None   0
8     8        GridFins  INTEGER        0       None   0
9     9          Reused  INTEGER        0       None   0
10   10            Legs  INTEGER        0       None   0
11   11      LandingPad     TEXT        0       None   0
12   12           Block     REAL        0       None   0
13   13     ReusedCount  INTEGER        0       None   0
14   14          Serial     TEXT        0       None   0
15   15       Longitude     REAL        0       None   0
16   16 

In [20]:
df = pd.read_csv('dataset_part_2.csv')
df.to_sql("SPACEXTBL", con=engine, if_exists='replace', index=False, method="multi")

90

In [21]:
query = 'SELECT DISTINCT "LaunchSite" FROM SPACEXTBL;'
df = pd.read_sql_query(query, con)
print("Unique Launch Sites:\n", df)

Unique Launch Sites:
      LaunchSite
0  CCSFS SLC 40
1   VAFB SLC 4E
2    KSC LC 39A


In [22]:
query = "PRAGMA table_info(SPACEXTBL);"
df_columns = pd.read_sql_query(query, con)
column_names = df_columns['name'].tolist()

# Find the launch site column dynamically
launch_site_column = next((col for col in column_names if "launch" in col.lower() and "site" in col.lower()), None)

if launch_site_column:
    # Query to get 5 records where launch site starts with 'CCA'
    query = f'SELECT * FROM SPACEXTBL WHERE "{launch_site_column}" LIKE \'CCA%\' LIMIT 5;'
    df = pd.read_sql_query(query, con)
    print("Records where launch site starts with 'CCA':\n", df)
else:
    print("No column found that likely represents the launch site.")
    

Records where launch site starts with 'CCA':
 Empty DataFrame
Columns: [FlightNumber, Date, BoosterVersion, PayloadMass, Orbit, LaunchSite, Outcome, Flights, GridFins, Reused, Legs, LandingPad, Block, ReusedCount, Serial, Longitude, Latitude, Class]
Index: []


In [23]:
# Get column names from the table
query = "PRAGMA table_info(SPACEXTBL);"
df_columns = pd.read_sql_query(query, con)
column_names = df_columns['name'].tolist()

# Find the payload mass column
payload_mass_column = next((col for col in column_names if "payload" in col.lower() and "mass" in col.lower()), None)

# Find the customer or mission column
customer_column = next((col for col in column_names if "customer" in col.lower() or "mission" in col.lower()), None)

if payload_mass_column and customer_column:
    # Query to get the total payload mass for NASA CRS missions
    query = f'''
    SELECT SUM("{payload_mass_column}") AS Total_Payload_Mass
    FROM SPACEXTBL
    WHERE "{customer_column}" LIKE '%NASA%CRS%';
    '''
    df = pd.read_sql_query(query, con)
    total_payload_mass = df['Total_Payload_Mass'].iloc[0]
    print(f"Total Payload Mass for NASA (CRS) Launches: {total_payload_mass} kg")
else:
    print("Error: Could not find the necessary columns for payload mass or customer/mission type.")



Error: Could not find the necessary columns for payload mass or customer/mission type.
